# Reinforcement Learning from Human Feedback (RLHF)

If we recap the Generative AI lifecycle, in the "Adapt and Align Model" Section we have three sub-sections
- Prompt Engineering *(Completed)*

- Fine tuning *(Completed)*
- Align with Human Feedback **(We are here)**

We have tried both the ways Prompt Engineering and Fine-tuning to optimize our model responses and produce the outputs in a more human understandable way. However, there are cases while the model is behaving badly

## Models behaving badly

Consider there are three cases, where the following questions are asked to the LLM model: 

- Asked to provide a Joke? - Replied with some non-related answer. **(Was it Helpful?)** 

- Asked some health related question - Replied with a known answer which the model should have refused. **(Was it Honest?)**

- Asked some harmful or dangerous question - Replied with steps instead of refusing **(Was it Harmlessness?)** 

These imporant human values - Helpfulness, Honesty and Harmlessness are sometimes combinely called HHH and these are set of principles which guide developers in use of responsible AI.

Additional fine-tuning with Human feedback helps to better align models with human preferences and helps in increasing the HHH values and helps in decreasing the toxicity.

## How RLHF works?

- It was proved by the researchers that when the LLM model which was finetuned with RLFH has surpassed the performance of all other models like pre-trained model, Instruction fine-tuned models.

- It uses the method of Reinforcement Learning Technique to fine-tune the model based on the human feedback. 

- The whole process in simple steps: 

    Instruct fine-tuned model -> RLHF -> Human feedback aligned LLM

    As the name suggests the RLHF will help to maximize the helpfulness relevance, minimize harm and avoids dangerous topics.
    
- Models will learn the preference of each individual users through personal feedback. This will help in personalization of LLMs leading to a significant development of Personal AI Assistants, or Individualized learning plans. 

### Understanding Process

![RLFH Process](https://miro.medium.com/v2/resize:fit:1400/1*PK8BewZA0daezu-rZWm_Wg.png)

<a href="https://miro.medium.com/v2/resize:fit:1400/1*PK8BewZA0daezu-rZWm_Wg.png">Image Credits</a>

In the Reinforcement Learning Technique, we have two main entities named "Agent" and "Environment". The Agent will be assigned with some RL policy and Environment is the place where the model will work. We also have action and state, and reward connected to each other. 

Similarly the above diagram depicts the Reinforcement Learning from Human Feedback in terms of LLM. The following are the steps involved: 

**Step 1:** Agent is our Instruct LLM, and will be assigned the LLM policy. 

**Step 2:** Once the agent (LLM) is provided with a prompt, then the action will be triggered and sent to the Environment (LLM Context).

**Step 3:** Environment produces the response and it returns the current context state back to the Agent. 

**Step 4:** The crucial part here is there will be Human labelers or Reward model which evaluates the response generated by the LLM and checks if the response is less harmful, honest and helpful. Based on some score gets calculated and sent as form of "Reward" to the Agent.

**Step 5:** Likewise this process repeats, so that LLM will learn from the Human feedback for each response it is generated. 

The value of the reward can lie between 0 and 1 which will be provided by the Human after keenly evaluating the response. Most often it is difficult for a human to manually evaluate the responses, hence we can also use "Reward Model" which is trained using Supervised Machine Learning technique aganist dataset of various prompts and completions. 

This Reward model will act like a human observer, and provides the output to the end users. Now as we understood the high-level steps of how RLFH works, lets deep dive in further each step in-detail. 

## Replacing Humans with Reward Model

### Obtaining Feedback from Human labelers and gathering data

This is the initial step in the process of RLHF where we have the Human labelers providing the feedback for the completions that are generated by our LLM. 

> Remember, we are using an already fine-tuned LLM which was trained on multiple tasks, and we are passing our sample dataset to test the Toxicity, Hate, etc. completions from the LLM.

Now, the Human labelers will review the each and every completion aganist the prompt entry that was generated by the LLM and provide the 'Rank' accordingly. 

#### Example for a Single prompt completion scenario

**Prompt:** My house is too hot.

**Completions[0]:** It is not hot.

**Completions[1]:** You can use Air-conditioning to cool.

**Completions[2]:** It is now common everywhere

| Completions | Human Labeler - 0 | Human Labeler - 1 | Human Labeler - 2 |
| :-----: | :-----: | :-----: | :-----: | 
| It is not hot | 3 | 3 | 2 |
| You can use Air-conditioning to cool. | 1 | 1 | 3 |
| It is now common everywhere | 2 | 2 | 1 |

In the above table; 1, 2, 3 are the "Rank's" assigned by each Human labeler based on how suitable the completion is for the users. 1 means good response and 3 being the bad response. But if we carefully observe one Human labeler provided in a wrong way.

The same set of validation is done by lots of Human labelers and there are many chances that they may validate it incorrectly. In most of the cases, these Human Labelers are drawn out randomly from a set of population and provided with some rules on assigning the rank for the completions generated by the LLM. 

Now, once we get the complete data we will train our "Reward Model" instead of using the human labelers. We can leverage the supervised learning technique to train our model. We will discuss more on this in the next step. 

### Preparing labeled data for training the Reward Model

Once the human labelers labels the data completely we will have a dataset comprising of the completions along with the ranks provided by many human labelers.

But we can't use the same dataset, to train our Reward model using Supervided learning. Hence we need to pre-process the dataset using a technique where the ranks are need to be converted into pairs. 

Refer the below example while we are converting one human labeler rank into a pair wise data.

| Completion Ranks | Pairs [$\ y_j$, $\ y_k$] | 
| :----: | :----: | 
| 2 | [1,0] | 
| 3 | [0,1] |  
| 1 | [1,0] | 

If we observe the above table, as we mentioned before, Ranks 1 and 2 are considered most correct and 3 means the incorrect answer. So, the we will re-arrange the responses making the most appropriate responses on the top and inappropriate in the bottom. 

In short, we are telling the model to choose the top response. 

| Completion Ranks | Pairs [$\ y_j$, $\ y_k$] | Final Rankings | 
| :----: | :----: | :----: | 
| 2 | [1,0] | [1,0] -> 2 | 
| 3 | [0,1] | [1,0] -> 1 | 
| 1 | [1,0] | [0,1] -> 3 |

where 
- $\ y_j$ means model knows whether the response should be chosen or not (preferred completion)
- $\ y_k$ it is the negation of the above parameter. Model will try not to prefer this completion. 

### Training the Reward model

Now, since our dataset is ready the final step is to Train the Reward model using supervised Machine Learning Technique. 

> Remember, this Reward model is also a language model.

It works based on the provided statement: For a given completion ($\ x$), and the completions $\ y_j$, $\ y_k$ the reward model will be able to predict the rewards $\ r_j$ and $\ r_k$ respectively. 

Based on this we will be able to calculate the Loss Function = $\ log(\sigma( r_j - r_k )) $

Instead of Humans providing the reward, now the reward model will act like a "Binary Classifier" to provide reward value for each prompt completion pair. 

For example: 

Prompt: -------

Completion: --------

Passed above (prompt, completion) parameters to Reward model. Consider our reward model predicts hate 

**Completion Case 1:** This dish tastes very good.

**Output:** 
| Class type | Value |
| :----: | :-----: |
| +ve class | +5.0 |
| -ve class | -2.0 |

**Completion Case 2:** This dish tastes the worst

**Output:** 
| Class type | Value |
| :----: | :-----: |
| +ve class | -1.01 |
| -ve class | +0.18 |

## Single Iteration Process using Reward Model

1. As we know already, we are using a Instruction fine-tuned Functional Model as our LLM which we are going to use it for RLHF.

2. From our prompt dataset, one prompt will be passed to our LLM, then an completion gets generated. 
3. Both the (prompt, completion) will be considered as a key-value pair and will be passed to our Reward Model.
4. The Reward model will evaluate both the prompt and completion and based on the data that is trained upon, it will predict the reward value (+ve class and -ve class). 
5. Then this Reward value will be sent to the Reinforcement Learning (RL) Algorithm/Proximal Policy Optimization (PPO) which will update the weights of the LLM. 
6. This same process will be repeated until the Reward score gets increased and to make the "Fine-tuned LLM" to "Human-aligned LLM".


### Proximal Policy Optimization (PPO) or RL Algorithm

This is the algorithm which will take the value predicted by the Reward model, and updates the weights of the LLM which helps in improving the reward score. 

Here as per the terminology, LLM is our policy so the algorithm that we are using PPO is optimizing the policy i.e., LLM.

## Potential Problem: Reward Hacking

This is a potential problem occurs during the RLHF process. 

The Agent (Fine-tuned LLM) tries to cheat the system by favouring actions that "maximizes the reward score received even if the response doesn't align well with the original objective". 

E.g., 

**Prompt:** A product is ...

**Completion:** most awesome and beautiful 

**Reward Score (predicted):** 2.6 (high even though responses is not aligned)

The agent(LLM) is trying to provide exaggerated response to make the Reward Score look high. 

### How to overcome this problem?

To solve this potential issue, we need to leverage the Fine-tuned LLM as our reference. The original prompt will be sent simultaneosuly to both the LLMs (Fine-tuned and Human-aligned). 

Now once we receive the completions we will then use the help of "KL Divergence Penalty Score" which helps in calculating the difference between the probability distributions of both the completions generated by the LLMs. 

Then the KL divergence will produce a penalty if the response provided is incorrect which will be sent to our RL Algorithm or PPO.